<a href="https://colab.research.google.com/github/danm91/le_kingmakers/blob/master/working_version_distilbert%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 2.3MB 13.0MB/s 
     |████████████████████████████████| 901kB 51.5MB/s 
     |████████████████████████████████| 3.3MB 57.4MB/s 


In [2]:
pip install ktrain

     |████████████████████████████████| 25.3MB 153kB/s 
     |████████████████████████████████| 6.8MB 45.1MB/s 
     |████████████████████████████████| 983kB 48.1MB/s 
     |████████████████████████████████| 266kB 63.5MB/s 
     |████████████████████████████████| 1.9MB 47.7MB/s 
     |████████████████████████████████| 1.2MB 48.9MB/s 
     |████████████████████████████████| 471kB 65.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=306595aaeeaa4e7dac777bf2d85cfce43e2a57f8074e6211f98fb6426571c7b8
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=92d464df4e8af48c2db223a6bd9f906cee18bc9c8b6aab0da39474b9c2058a4e
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
  Created wheel for syntok: filename=syntok-1.3.1-cp37-none-any.whl size=20919 

In [3]:
import ktrain
from ktrain import text
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Le KingMakers/kaggle.csv', names =['target', '2', '3', '4', '5', 'text'], encoding='latin-1')

In [6]:
df = df.sample(frac=0.005, random_state=0).copy()
df['target'] = df['target'].map({4:1,0:0})
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['target'],test_size=0.2, stratify=df['target'])
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25,stratify=y_train)

In [7]:
model_name = 'distilbert-base-uncased'
transformer = text.Transformer(model_name,maxlen=30, class_names=['negative', 'positive'])

In [8]:
train_data = transformer.preprocess_train(np.array(X_train), np.array(y_train))
val_data = transformer.preprocess_test(np.array(X_val),np.array(y_val))

preprocessing train...
language: en
train sequence lengths:
	mean : 13
	95percentile : 25
	99percentile : 28


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 13
	95percentile : 26
	99percentile : 28


In [9]:
model = transformer.get_classifier()

In [10]:
learner = ktrain.get_learner(model,train_data=train_data, val_data=val_data, batch_size=8)

In [11]:
learner.lr_find(show_plot=True,max_epochs=2)

simulating training for different learning rates... this may take a few moments...
Epoch 1/2
 22/600 [>.............................] - ETA: 11:05 - loss: 0.6942 - accuracy: 0.5114

In [20]:
learner.fit(lr=0.005,n_cycles=2)

Epoch 1/2
  9/600 [..............................] - ETA: 11:22 - loss: 0.7801 - accuracy: 0.5417

KeyboardInterrupt: ignored

In [ ]:
learner.validate(class_names=['negative', 'positive'])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc = transformer)

In [ ]:
predictor.predict('i hate you')